In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression

In [2]:
training = pd.read_parquet('../data/curated/clean_stats_13-22')

In [3]:
training.set_index('player_id', inplace=True)
training

,match_id,match_home_team,match_away_team,match_date,match_round,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,match_away_team_behinds,...,f50_ground_ball_gets_total,ground_ball_gets_total,hitouts_to_advantage_total,hitout_win_percentage_total,intercept_marks_total,shots_at_goal_total,spoils_total,coaches_votes,prev_total_bv,brownlow_votes
player_id,,,,,,,,,,,,,,,,,,,,,
10398,14167,Adelaide,Essendon,2013-03-22,1,11,16,82,18,9,...,36,197,21,164,27,53,69,1.0,1.0,0.0
11139,14167,Adelaide,Essendon,2013-03-22,1,11,16,82,18,9,...,36,197,21,164,27,53,69,0.0,23.0,0.0
11177,14167,Adelaide,Essendon,2013-03-22,1,11,16,82,18,9,...,36,197,21,164,27,53,69,0.0,1.0,0.0
11234,14167,Adelaide,Essendon,2013-03-22,1,11,16,82,18,9,...,36,197,21,164,27,53,69,0.0,0.0,0.0
11272,14167,Adelaide,Essendon,2013-03-22,1,11,16,82,18,9,...,36,197,21,164,27,53,69,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12863,16346,St Kilda,Sydney,2022-08-21,23,11,8,74,13,10,...,37,197,21,149,31,53,66,0.0,0.0,0.0
12864,16346,St Kilda,Sydney,2022-08-21,23,11,8,74,13,10,...,37,197,21,149,31,53,66,0.0,2.0,0.0
12945,16346,St Kilda,Sydney,2022-08-21,23,11,8,74,13,10,...,37,197,21,149,31,53,66,0.0,0.0,0.0


In [4]:
predicting = pd.read_parquet('../data/curated/stats_2023_final')

In [5]:
predicting.set_index('player_id', inplace=True)
predicting.head()

,match_id,player_first_name,player_last_name,team_name,coaches_votes,afl_fantasy_score,disposals_proportion,rating_points,contested_possessions_proportion,score_involvements_proportion,...,score_involvements,kicks_proportion,clearances_proportion,clearances,metres_gained_proportion,metres_gained,handballs_proportion,prev_total_bv,uncontested_possessions_proportion,goals
player_id,,,,,,,,,,,,,,,,,,,,,
CD_I1000223,10007,Liam,Baker,Richmond,0,55,0.021739,5.7,0.013468,0.027586,...,4,0.022272,0.000000,0,0.022284,273,0.020747,6.0,0.025445,0
CD_I1002245,10007,Noah,Balta,Richmond,0,35,0.011594,7.2,0.016835,0.006897,...,1,0.013363,0.000000,0,0.016244,199,0.008299,2.0,0.005089,0
CD_I993993,10007,Shai,Bolton,Richmond,0,88,0.026087,7.9,0.026936,0.020690,...,3,0.033408,0.044118,3,0.046282,567,0.012448,12.0,0.030534,1
CD_I295203,10007,Nathan,Broad,Richmond,0,70,0.018841,4.1,0.006734,0.006897,...,1,0.024499,0.014706,1,0.017060,209,0.008299,1.0,0.027990,0
CD_I270896,10007,Trent,Cotchin,Richmond,0,57,0.026087,8.1,0.037037,0.020690,...,3,0.020045,0.058824,4,0.021468,263,0.037344,5.0,0.017812,0


In [13]:
actual_features = [  
    'coaches_votes',
    'afl_fantasy_score',
    'disposals_proportion',
    'rating_points',
    'contested_possessions_proportion',
    'score_involvements_proportion',
    'effective_disposals_proportion',
    'contested_possessions',
    'effective_disposals',
    'score_involvements',
    'kicks_proportion',
    'clearances_proportion',
    'clearances',
    'metres_gained_proportion',
    'metres_gained',
    'handballs_proportion',
    'prev_total_bv',
    'uncontested_possessions_proportion',
    'goals'
]

In [14]:
X_train, y_train = training[actual_features], training['brownlow_votes']
X_test = predicting[actual_features]

In [15]:
# creating variable to store player id's
y_test = predicting['player_last_name']

In [16]:
LR = LogisticRegression(max_iter=5_000, multi_class='multinomial', solver='newton-cg')

In [17]:
LR.fit(X_train, y_train)

LogisticRegression(max_iter=5000, multi_class='multinomial', solver='newton-cg')

In [18]:
cols = [
    'match_id', 'player_first_name', 'player_last_name', 'team_name',
]

In [19]:
prob_2023 = pd.DataFrame(LR.predict_proba(X_test))
prob_2023['player_id'] = y_test.index
prob_2023.set_index('player_id', inplace=True)

In [20]:
prob_2023

,0,1,2,3
player_id,,,,
CD_I1000223,0.996487,0.002483,0.000929,0.000102
CD_I1002245,0.999040,0.000734,0.000210,0.000016
CD_I993993,0.958959,0.024597,0.013378,0.003067
CD_I295203,0.996505,0.002441,0.000952,0.000102
CD_I270896,0.993833,0.004317,0.001632,0.000219
...,...,...,...,...
CD_I294305,0.602763,0.146600,0.152148,0.098489
CD_I1008139,0.991778,0.005587,0.002312,0.000323
CD_I290675,0.988751,0.007658,0.003161,0.000431


In [21]:
df_2023 = predicting[cols]

In [22]:
final_df = pd.concat([df_2023, prob_2023], axis=1)

In [23]:
final_df.head()

,match_id,player_first_name,player_last_name,team_name,0,1,2,3
player_id,,,,,,,,
CD_I1000223,10007,Liam,Baker,Richmond,0.996487,0.002483,0.000929,0.000102
CD_I1002245,10007,Noah,Balta,Richmond,0.999040,0.000734,0.000210,0.000016
CD_I993993,10007,Shai,Bolton,Richmond,0.958959,0.024597,0.013378,0.003067
CD_I295203,10007,Nathan,Broad,Richmond,0.996505,0.002441,0.000952,0.000102
CD_I270896,10007,Trent,Cotchin,Richmond,0.993833,0.004317,0.001632,0.000219


In [24]:
final_df.to_parquet('../data/curated/2023_prob')

/Users/rileymacdonald/miniconda3/envs/afl/lib/python3.11/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
